In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('../Assignment_MIMIC_SQL/database.db')

# Fetch ICU stays and mortality labels
query_icu = """
SELECT icu.SUBJECT_ID, icu.HADM_ID, icu.ICUSTAY_ID, 
       adm.ADMITTIME, adm.DEATHTIME, 
       CASE 
           WHEN adm.DEATHTIME IS NOT NULL AND adm.DEATHTIME <= icu.OUTTIME THEN 1 
           ELSE 0 
       END AS MORTALITY
FROM ICUSTAYS icu
JOIN ADMISSIONS adm ON icu.HADM_ID = adm.HADM_ID;
"""
icu_stays = pd.read_sql(query_icu, conn)

# Fetch vital signs
query_vitals = """
SELECT ce.SUBJECT_ID, ce.HADM_ID, ce.ICUSTAY_ID, 
       ce.ITEMID, ce.CHARTTIME, ce.VALUE
FROM CHARTEVENTS ce
WHERE ce.ITEMID IN (211, 220045, 220179, 220180, 220210, 220277, 223761)
AND ce.VALUE IS NOT NULL;
"""
vitals = pd.read_sql(query_vitals, conn)

# Fetch lab test results
query_labs = """
SELECT le.SUBJECT_ID, le.HADM_ID, 
       le.ITEMID, le.CHARTTIME, le.VALUE
FROM LABEVENTS le
WHERE le.ITEMID IN (50809, 50912, 51300)
AND le.VALUE IS NOT NULL;
"""
labs = pd.read_sql(query_labs, conn)

# Close the connection
conn.close()

# Display sample data
print("ICU Stays:\n", icu_stays.head())
print("Vitals:\n", vitals.head())
print("Labs:\n", labs.head())



ICU Stays:
    SUBJECT_ID  HADM_ID  ICUSTAY_ID            ADMITTIME            DEATHTIME  \
0         268   110404      280836  2198-02-11 13:40:00  2198-02-18 03:55:00   
1         269   106296      206613  2170-11-05 11:04:00                 None   
2         270   188028      220345  2128-06-23 18:26:00                 None   
3         271   173727      249196  2120-08-07 18:56:00                 None   
4         272   164716      210407  2186-12-25 21:06:00                 None   

   MORTALITY  
0          1  
1          0  
2          0  
3          0  
4          0  
Vitals:
    SUBJECT_ID  HADM_ID  ICUSTAY_ID  ITEMID            CHARTTIME  VALUE
0          36   165660    241249.0  220045  2134-05-12 13:00:00   86.0
1          36   165660    241249.0  220179  2134-05-12 13:00:00  137.0
2          36   165660    241249.0  220180  2134-05-12 13:00:00   72.0
3          36   165660    241249.0  220210  2134-05-12 13:00:00   21.0
4          36   165660    241249.0  220277  2134-05-1

In [ ]:
# 1. Convert Timestamps to datetime Format
# This ensures we can sort and resample data correctly.
icu_stays["ADMITTIME"] = pd.to_datetime(icu_stays["ADMITTIME"])
icu_stays["DEATHTIME"] = pd.to_datetime(icu_stays["DEATHTIME"])

vitals["CHARTTIME"] = pd.to_datetime(vitals["CHARTTIME"])
labs["CHARTTIME"] = pd.to_datetime(labs["CHARTTIME"])


In [ ]:
# 2. Map ITEMID to Readable Feature Names
# Since ITEMID numbers aren't human-readable, we map them to their respective vitals/labs.
vital_signs_mapping = {
    211: "HEART_RATE",
    220045: "MEAN_BP",
    220179: "SYSTOLIC_BP",
    220180: "DIASTOLIC_BP",
    220210: "RESP_RATE",
    220277: "SPO2",
    223761: "TEMPERATURE"
}

lab_tests_mapping = {
    50809: "GLUCOSE",
    50912: "CREATININE",
    51300: "WBC_COUNT"
}

# Replace ITEMID with actual feature names
vitals["ITEMID"] = vitals["ITEMID"].map(vital_signs_mapping)
labs["ITEMID"] = labs["ITEMID"].map(lab_tests_mapping)


In [6]:
# Pivot the tables so each ITEMID becomes a column
labs["VALUE"] = pd.to_numeric(labs["VALUE"], errors="coerce")
vitals["VALUE"] = pd.to_numeric(vitals["VALUE"], errors="coerce")



In [ ]:
# Pivot the tables so each ITEMID becomes a column
vitals_pivot = vitals.pivot_table(index=["SUBJECT_ID", "HADM_ID", "CHARTTIME"], 
                                  columns="ITEMID", values="VALUE").reset_index()

# Pivot the tables so each ITEMID becomes a column
labs_pivot = labs.pivot_table(index=["SUBJECT_ID", "HADM_ID", "CHARTTIME"], 
                              columns="ITEMID", values="VALUE").reset_index()


In [10]:
# Merge vitals and labs
merged_data = vitals_pivot.merge(labs_pivot, on=["SUBJECT_ID", "HADM_ID", "CHARTTIME"], how="outer")

# Merge with ICU stay data (Mortality Label)
final_data = merged_data.merge(icu_stays[["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID", "MORTALITY"]], 
                               on=["SUBJECT_ID", "HADM_ID"], how="inner")

# Sort by time
final_data = final_data.sort_values(by=["SUBJECT_ID", "CHARTTIME"])


In [11]:
# Fill missing values forward and backward for vitals
final_data.fillna(method="ffill", inplace=True)
final_data.fillna(method="bfill", inplace=True)

# Impute remaining missing values with median
final_data.fillna(final_data.median(), inplace=True)


In [12]:
from sklearn.preprocessing import StandardScaler

# Select numeric columns (excluding SUBJECT_ID, HADM_ID, CHARTTIME)
feature_cols = [col for col in final_data.columns if col not in ["SUBJECT_ID", "HADM_ID", "CHARTTIME", "ICUSTAY_ID", "MORTALITY"]]

# Scale features
scaler = StandardScaler()
final_data[feature_cols] = scaler.fit_transform(final_data[feature_cols])


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert Data to LSTM/GRU Format
window_size = 24  

def create_sequences(df, window_size):
    sequences, labels = [], []
    for patient_id in df["ICUSTAY_ID"].unique():
        patient_data = df[df["ICUSTAY_ID"] == patient_id].sort_values("CHARTTIME")
        for i in range(len(patient_data) - window_size):
            seq = patient_data.iloc[i : i + window_size][feature_cols].values
            label = patient_data.iloc[i + window_size]["MORTALITY"]  # Binary outcome
            sequences.append(seq)
            labels.append(label)
    return np.array(sequences), np.array(labels)

# Convert to sequences
X, y = create_sequences(final_data, window_size)

# Split into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save the datasets as .npy files
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)

print("Final Shape:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print("Dataset saved successfully!")

Final Shape: (7413697, 24, 10) (1853425, 24, 10) (7413697,) (1853425,)
Dataset saved successfully!


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(64, return_sequences=False),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    return model

# Example usage (assuming X_train is preprocessed and shaped correctly)
model = build_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))
model.summary()

NameError: name 'X_train' is not defined

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load your preprocessed dataset
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_val = np.load("X_test.npy") 
y_val = np.load("y_test.npy")


# Define the LSTM model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_lstm_model.h5', save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=2,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]
)

# Save final model
model.save("final_lstm_model.h5")


2025-03-15 10:44:46.009524: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-03-15 10:44:46.010003: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-03-15 10:44:46.010016: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2025-03-15 10:44:46.010958: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-15 10:44:46.011409: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2025-03-15 10:46:50.953367: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-03-15 10:46:51.223362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-03-15 10:46:51.772747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-03-15 10:46:52.877170: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-03-15 10:46:53.470697: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


231679/231679 [==============================] - ETA: 0s - loss: 0.2806 - accuracy: 0.8990

2025-03-15 11:27:22.552159: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-03-15 11:27:22.613854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-03-15 11:27:22.647874: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


231679/231679 [==============================] - 2664s 11ms/step - loss: 0.2806 - accuracy: 0.8990 - val_loss: 0.2550 - val_accuracy: 0.9043
Epoch 2/50
    11/231679 [..............................] - ETA: 43:11 - loss: 0.2282 - accuracy: 0.9176

/opt/homebrew/Caskroom/miniconda/base/envs/ai_health/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


231679/231679 [==============================] - 2741s 12ms/step - loss: 0.2355 - accuracy: 0.9103 - val_loss: 0.2199 - val_accuracy: 0.9154
Epoch 3/50
231679/231679 [==============================] - 2742s 12ms/step - loss: 0.2106 - accuracy: 0.9186 - val_loss: 0.2019 - val_accuracy: 0.9216
Epoch 4/50
231679/231679 [==============================] - 2746s 12ms/step - loss: 0.1975 - accuracy: 0.9229 - val_loss: 0.1934 - val_accuracy: 0.9242
Epoch 5/50
231679/231679 [==============================] - 2748s 12ms/step - loss: 0.1893 - accuracy: 0.9257 - val_loss: 0.1844 - val_accuracy: 0.9276
Epoch 6/50
212273/231679 [==========================>...] - ETA: 3:31 - loss: 0.2072 - accuracy: 0.9195

KeyboardInterrupt: 

In [5]:
import matplotlib.pyplot as plt

# Extract history data
history_dict = history.history

# Plot Loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_dict['loss'], label='Training Loss')
plt.plot(history_dict['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_dict['accuracy'], label='Training Accuracy')
plt.plot(history_dict['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')

plt.show()


NameError: name 'history' is not defined